# __Block and Stream Ciphers__
### __Planning and Progress__

#### __Test Cases needed:__
* cipherType: B or S Error: "Invalid Function Type" return 1
* inputFile path does not exist "Input File Does Not Exist" return 1
* * Key length validation:
  - Block cipher: key must be exactly 16 bytes
  - Stream cipher: key can be any length
  - If key length wrong for block cipher -> silent exit (return 1)
* KeyFile path (16) "Key File Does Not Exist"
* Mode of Ops: 'D' or 'E' Error: Invalid Mode Type
* Newlines or EOL treated like any other character, program should preserve newlines in same order 
* if input is empty the output file should be empty

#### __Encryption Block Cipher__

- Pad
- Encrypt XOR
- Swap

__Block Size: 16 bytes__

__Read:__

- whole input string from input file, store in a vector take 16 byte substring as blocks until read whole string
OR
- read in 16 byte blocks as chunks from input file at a time until EOF

__Pad:__

if < 16 bytes
0x81 

__XOR:__
XOR the 16 byte data block with 16-byte key in bytewise manner
each byte starting from leftmost byte 0 index will be XOR'd with each 16-byte data block
for i = 0, i++

__Swap:__
swap bytes for each XOR'd output
from (start) 0 -> n(end), each being a pointer of XOR'd outblock 
for reach byte of the key starting from 0 calculate:

ASCII value of byte % 2 = 0 or 1

if val = 0:
- do not swap anything and start++
if val =1:
swap byte position pointed to by start pointer with byte position pointed to by the end pointer
start++
end--

if key is exhausted- restart from first byte of key index 0
process is carried on until the start and end pointers collide:
if start == end:
stop

then it produces the output

__Decryption:__
- swap
- decrypt XOR
- remove padding if needed

because the D process occurs in reverse direction
cannot use same module ot perform both


Since I will be working with raw binary data, bytes, (encryption deals with bytes 0-255) i should use unsigned char.

the functions will be taking in data and returning encrypted data, so no object creation is necessary- functions can be static like BlockCipher cipher; cipher.encrypt(data, key);

for secure parameters, const will be used: const<br>
my example: std::vector<unsigned char>& input<br>
passing by reference &<br>

in Block, i need a minimum value returned, <cmath> can't do this, found <algorithm> and it can for this data type so im going to use that and hopefully it will work :)



#### __Stream Cipher Encryption__
XOR input stream with the key:
- the key will be given in ASCII > 16
- XOR bytes or char from input file with bytes or char from key file starting from 0
- byte 0 from input file will be XOR'd with byte 0 from key
- byte 1 from input will be XOR'd with bit 1 from key
- byte n-1 from input will be XOR'd with bit 1 from key
- if key is exhausted- restart from first byte of key index 0
__once key end is reached start over from index 0__

__Encryption and Decryption__

performed with same module as stream cipher


# Utility Class

## Research and Planning

**Utility**

1. existingFile(filename)
 * First: Make sure there are exactly 5 arguments (argc == 6)
       - If not: exit silently (no error message)
     
     * Second: Save arguments into the reference variables
       - cipherType = argv[1]  ("B" or "S")
       - inputFile = argv[2]    (path to input file)
       - outputFile = argv[3]   (path to output file)
       - keyFile = argv[4]      (path to key file)
       - mode = argv[5]         ("E" or "D")
     
     * Third: Check cipherType is "B" or "S"
       - If not: print "Invalid Function Type" and exit
     
     * Fourth: Check input file exists (call existingFile)
       - If not: print "Input File Does Not Exist" and exit
     
     * Fifth: Check key file exists (call existingFile)
       - If not: print "Key File Does Not Exist" and exit
     
     * Sixth: Check mode is "E" or "D"
       - If not: print "Invalid Mode Type" and exit
     
     * Seventh: All good, return true

   - Note: Key length NOT checked here (key file not read yet)
   - Note: Only the 4 exact error messages from PDF are printed

3. **readFile(filename)**
   - Read entire file into memory as bytes
   - Steps:
     * Open file in binary mode (to preserve newlines)
     * Find out how big the file is
     * Create a vector of unsigned chars that size
     * Read the whole file into the vector
     * Close file and return the vector
   - If file can't be opened: exit silently

4. **writeFile(filename, data)**
   - Write vector of bytes to a file
   - Steps:
     * Open file in binary mode
     * Write all bytes from vector to file
     * Close file
   - If file can't be created: exit silently
   - If data is empty: still create empty file (PDF requirement)

**Important notes for all functions:**
- NO printing except the 4 exact error messages
- Binary mode for all file operations (preserves newlines)
- Use unsigned char for byte data (0-255 range)
- Silent exit(1) for any error not in the 4 specified messages

### Utility.h 

guards:

#ifndef UTILITY_H <br>
#define UTILITY_H <br>
#endif

Argument Validation

In [ ]:
bool validate(int argc, char* argv[],
              std::string& cipherType,
              std::string& mode,
              std::string& inputFile,
              std::string& outputFile,
              std::String& keyFile);

File Reading

In [ ]:
std::vector<unsigned char> readFile(const std::string& filename);

File Writing

In [ ]:
void writeFile(const std::string& filename, const std::vector<unsigned char>& data);

### Block planning
#### Block.h 


#### __Block.h Planning__

**Purpose:** 
block cipher functions  main.cpp can use them
create a class for block and stream so they can be called easily
```
BlockCipher
├── Public (for main to call)
│   ├── encrypt()
│   └── decrypt()
└── Private (internal helpers)
    ├── pad()
    ├── unpad()
    ├── xor_with_key()
    └── swapBytes()
```
**What this header does:**
- Declares encryption and decryption functions
- Defines constants (block size = 16, padding byte = 0x81)
- Other files include this to know what functions exist

**Constants:**
- `BLOCK_SIZE = 16` - block cipher works in 16-byte chunks
- `PAD_BYTE = 0x81` - padding value (outside ASCII range)

**Public functions (what main.cpp calls):**
1. `encrypt(input, key)` - takes plaintext, returns ciphertext
2. `decrypt(input, key)` - takes ciphertext, returns plaintext

**Private helper functions (used inside block.cpp only):**
1. `pad(block)` - adds 0x81 bytes to make block 16 bytes
2. `unpad(block)` - removes 0x81 bytes from end
3. `xorWithKey(block, key)` - XORs block with key bytewise
4. `swapBytes(block, key)` - swaps based on key parity (PDF algorithm)



In [ ]:
//interface
class BlockCipher{
public:
static std::vector<unsigned char> encrypt(const std::vector<unsigned char>& input,
const std::string& key);

static std::vector<unsigned char> decrypt(const std::vector<unsigned char>& input, const std::string& key)


private:
// Helper functions
static std::vector<unsigned char> pad(const std::vector<unsigned char>& block);
static std::vector<unsigned char> unpad(const std::vector<unsigned char>& block);
static std::vector<unsigned char> xorWithKey(const std::vector<unsigned char>& block,
const std::string& key);
static std::vector<unsigned char> swapBytes(const std::vector<unsigned char>& block,
const std::string& key);
};

#### __Block.cpp Planning__

**Purpose:** Implement all block cipher functions declared in block.h

**Function 1: pad(block)**

Input: block (vector of bytes, size ≤ 16) <br>
Output: padded block (exactly 16 bytes)<br>



1.  Make a copy of the input block

2.  While size < 16:

3.  Add PAD_BYTE (0x81) to the end

4. Return the padded block

Example:<br>
Input: [6c, 75] (2 bytes)<br>
Output: [6c, 75, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81]<br>


**Function 2: unpad(block)**

Input: block (vector of bytes, exactly 16 bytes from decryption)
Output: unpadded block (original plaintext)



1. Make a copy of the input block

2. While last byte == PAD_BYTE (0x81):

3. Remove the last byte

4. Return the unpadded block

Example:
Input: [6c, 75, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81, 81]
Output: [6c, 75]



**Function 3: xorWithKey(block, key)**
Input: block (16 bytes), key (16 bytes)
Output: XORed block (16 bytes)



1. 0Create result vector of same size

2. For i = 0 to block.size() - 1:

3. result[i] = block[i] ^ key[i % key.length()]

4. Return result

 i % key.length() makes key repeat 



**Function 4: swapBytes(block, key)**

Input: XORed block (16 bytes), key (16 bytes)
Output: Swapped block (16 bytes)



1. Make copy of block called swapped

2. Set start = 0, end = 15

3. Set keyIndex = 0

4. While start < end:

        If key[keyIndex] is odd (key[keyIndex] % 2 == 1):
        swap(swapped[start], swapped[end])
        start++
        end--
        Else:
        start++

        keyIndex = (keyIndex + 1) % key.length()

    Return swapped

Example from PDF with key "abcdefghijklmnop": <br>

start=0, end=15: key[0]='a'(97 odd) → swap bytes 0 and 15 <br>

start=1, end=14: key[1]='b'(98 even) → no swap, start=2 <br>

start=2, end=14: key[2]='c'(99 odd) → swap bytes 2 and 14 <br>

... continues until start >= end



**Function 5: encrypt(input, key)**

Input: entire input file (any size), key (16 bytes)
Output: encrypted ciphertext



1. Create empty result vector

2. For each 16-byte block in input:

    - Extract block (may be less than 16 bytes at end)

    - If block size < 16: pad it

    - XOR: xored = xorWithKey(block, key)

    - Swap: swapped = swapBytes(xored, key)

    - Append swapped to result

    - Return result

**Function 6: decrypt(input, key)**
Input: ciphertext (multiple of 16 bytes), key (16 bytes)
Output: decrypted plaintext (with padding removed)



1. Create empty result vector

2.  For each 16-byte block in input:

3. Extract block (always 16 bytes for ciphertext)

    - Swap: swapped = swapBytes(block, key)

    - XOR: xored = xorWithKey(swapped, key)

    - Append xored to result

    - Remove padding: result = unpad(result)

    - Return result

Note: Order is REVERSE of encryption!<br>
Encrypt: Pad → XOR → Swap<br>
Decrypt: Swap → XOR → Unpad<br>

In [ ]:

// INPUT: the encrypted ciphertext and the 16-byte key
// result holdsdecrypted data
std::vector<unsigned char> result


// LOOP: process the ciphertext in 16-byte chunks
// i starts at 0, then goes to 16, 32, 48... until the end
for (std::size_t i = 0; i < input.size(); i += BLOCK_SIZE) 


//extract one 16-byte block from the ciphertext

//  if i=0, we take bytes 0 through 15
//  if i=16, we take bytes 16 through 31
std::vector<unsigned char> block:
input.begin() + i,  //beginning
input.begin() + i + BLOCK_SIZE   // End at position i+16
// block now contains 16 bytes of ciphertext


// 1: Swap: reverse the swapping from encryption
std::vector<unsigned char> swapped = swapBytes(block, key)
// swapped now contains the block with bytes restored to pre-swap order


// 2: XOR - Reverse the XOR from encryption
// To decrypt, cipher XOR key = plain
std::vector<unsigned char> xored = xorWithKey(swapped, key)
// xored holds original plaintext

// 3: Append decrypted block result
// Take all decrypted bytes and add them to the end of result vector
result.insert
result.end()     // take til end of result
xored.begin()   
xored.end()    // Take until the end of xored
// result holds all decrypted


// STEP 4: Rremove padding,  from entire result
// unpad() will strip off all the 0x81 bytes from the end
return unpad(result)



### Stream

```
StreamCipher
└── Public for main to call
    └── process()  will enc and dec
```

Easy implementation compared to my block cipher since it requires less work with padding and checking for need to pad 
will create a class like i did with block 

create a check for empties and then process encryption or decryption 

loop through every byte of input until i <  the input size
for every i in the input XOR with the corresponding key byte (i % key length)

In [ ]:
Streamcipher::process(input,key)
if input is empty {return vector}

vector -> result(same as input size)
for ( first byte i = 0; until end; i++)
{
    result[i] = input[i] ^ key[i % key.length()]
}

return result